In [7]:
!pip install opencv-python numpy pyzbar mysql-connector-python ipywidgets qrcode
!pip install flask
!pip install python-dotenv
!pip install voilla

ERROR: Could not find a version that satisfies the requirement voilla (from versions: none)
ERROR: No matching distribution found for voilla


In [3]:
import cv2
import numpy as np
import pyzbar.pyzbar as pyzbar
import mysql.connector
import ipywidgets as widgets
from IPython.display import display, clear_output

In [29]:
import mysql.connector as sql
mydb=sql.connect(
   host="localhost",
   user="root",
   passwd="root",
   database="students")
print (mydb)

In [ ]:
#!jupyter nbextension enable --py widgetsnbextension --sys-prefix
#!jupyter serverextension enable voila --sys-prefix

In [4]:
from tkinter import *
from PIL import Image, ImageTk
from IPython.display import display, HTML

In [ ]:
import tkinter as tk
from tkinter import ttk
import qrcode
from tkinter import *
from PIL import Image, ImageTk
from IPython.display import display, HTML
import datetime
import mysql.connector
import cv2
import pyzbar.pyzbar as pyzbar
from pyzbar.pyzbar import decode
import io
from flask import Flask, request, session

root=Tk()
# set the window size
root.geometry("400x400")
# set the title of the window
root.title("QR Code Based Attendance Sytstem")
root.configure(background="navy blue")
label = tk.Label(root, text="LOG IN DASHBOARD")
label.pack(pady=10)
username_label = tk.Label(root, text="Username:")
username_label.pack(pady=10)
username_input = tk.Entry(root)
username_input.pack(pady=5)
password_label = tk.Label(root, text="Password")
password_label.pack()
password_input = tk.Entry(root, show="*")
password_input.pack()

app = Flask(__name__)
app.secret_key = 'secret-key'



# create a submit button to authenticate user inputs and assign privileges
def submit():
    conn = mysql.connector.connect(host="localhost", user="root", password="root", database="students")
    cursor = conn.cursor()   
    cursor.execute("SELECT * FROM coursedetails WHERE Instructor=%s AND InstructorID=%s", (username_input.get(), password_input.get()))
    instructor = cursor.fetchone()
    
    
    if instructor is not None:
        print("Welcome Instructor")
        # create a new window to display course details
        course_window = tk.Toplevel()
        course_window.title("Course Details")
        # create a label to display the course name and ID
        course_label = tk.Label(course_window, text="CourseName: {} \nCourseID: {}".format(instructor[0], instructor[1]))
        course_label.pack()

        # function to generate QR code
        def generate_qr_code(Course_ID):
            qr = qrcode.QRCode(version=1, box_size=10, border=5)
            #qr = qrcode.QRCode(version=1, box_size=10, border=5)
            qr.add_data(Course_ID)
            qr.make(fit=True)
            img = qr.make_image(fill_color="black", back_color="white")
            
            #Convert PIL image to PhotoImage
            byte_stream = io.BytesIO()
            img.save(byte_stream, format="PNG")
            byte_stream.seek(0)
            photo_image = tk.PhotoImage(data=byte_stream.getvalue())
          
            # Create new window and canvas to display the QR code image
            qr_window = tk.Toplevel()
            qr_canvas = tk.Canvas(qr_window, width=img.size[0], height=img.size[1])
            qr_canvas.pack()
            qr_canvas.create_image(0, 0, anchor="nw", image=photo_image)
        # create a button to generate QR code
        generate_qr_code_button = tk.Button(course_window, text="Generate QR Code", command=lambda: generate_qr_code(instructor[1]))
        generate_qr_code_button.pack()
       
        # function to view classlist
        def view_classlist():
            # create a new window to display the classlist
            classlist_window = tk.Toplevel()
            classlist_window.title("Classlist")
            frame = tk.Frame(classlist_window)
            frame.pack()
            
            # create a frame to hold the classlist
            classlist_frame = tk.Frame(classlist_window)
            classlist_frame.pack(fill=tk.BOTH, expand=True)
            # create a scrollbar for the student table
            scrollbar = tk.Scrollbar(classlist_frame)
            scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
            # create a table to display the student data
            classlist_table = ttk.Treeview(classlist_frame, columns=('RegNo', 'StudentsName', 'Attendance'), show='headings')
            classlist_table.pack(fill=tk.BOTH, expand=True)
            # configure the scrollbar and table widget to work together
            classlist_table.configure(yscrollcommand=scrollbar.set)
            scrollbar.configure(command=classlist_table.yview)
                
            # create a connection to the database
            conn = mysql.connector.connect(host="localhost", user="root", password="root", database="students")
            cursor = conn.cursor()
            
            # populate the classlist with data from the database
            cursor.execute("SELECT * FROM classlist")
            students = cursor.fetchall()
            
            for student in students:
                student_data = (student[0], student[1], student[2])
                classlist_table.insert('', tk.END, values=student_data)
                # set the column headings
                classlist_table.heading('RegNo', text='RegNo')
                classlist_table.heading('StudentsName', text='StudentsName')
                classlist_table.heading('Attendance', text='Attendance')
        view_classlist_button = tk.Button(course_window, text="View Classlist", command=view_classlist)
        view_classlist_button.pack()
            
        
    
        def update_classlist():
            classlist_window = tk.Toplevel()
            classlist_window.title("Update classlist")
            classlist_frame = tk.Frame(classlist_window)
            classlist_frame.pack(fill=tk.BOTH, expand=True)
            form_frame = tk.Frame(classlist_frame)
            form_frame.pack(pady=10)
            classlist_table = ttk.Treeview(classlist_frame, columns=('RegNo', 'StudentsName', 'Attendance','Rank'), show='headings')
            classlist_table.pack(fill=tk.BOTH, expand=True)
                
            regno_label = tk.Label(form_frame, text='RegNo')
            regno_label.grid(row=0, column=0)
            regno_entry = tk.Entry(form_frame)
            regno_entry.grid(row=0, column=1)
                
            studentsname_label = tk.Label(form_frame, text='StudentsName')
            studentsname_label.grid(row=1, column=0)
            studentsname_entry = tk.Entry(form_frame)
            studentsname_entry.grid(row=1, column=1)
                
                
            # define functions for adding students
                
            def add_student():
                classlist_frame = tk.Frame(classlist_window)
                classlist_frame.pack(fill=tk.BOTH, expand=True)
                form_frame = tk.Frame(classlist_frame)
                form_frame.pack(pady=10)
                regno = regno_entry.get()
                studentsname = studentsname_entry.get()
                if regno and studentsname:
                        
                    conn = mysql.connector.connect(host="localhost", user="root", password="root", database="students")
                    cursor = conn.cursor()
                    cursor.execute("INSERT INTO classlist (RegNo, StudentsName) VALUES ( %s, %s)", (regno, studentsname))
                    conn.commit()
                                   
                    student_data = (regno, studentsname)
                    classlist_table.insert('', tk.END, values=student_data)
                    
                    regno_entry.delete(0, tk.END)
                    studentsname_entry.delete(0, tk.END)
                else:
                    print('Error', 'Please enter all fields')
            def delete_student():
                reg_no=regno_entry.get()
                if reg_no:
                    sql= "DELETE FROM classlist WHERE RegNo = %s"
                    val= (reg_no,)
                    cursor.execute(sql,val)
                    conn.commit()
                    if cursor.rowcount > 0:
                            print(f"Student with registration number {reg_no} has been deleted from the classlist.")
                    else:
                        print(f"No student found with registration number {reg_no}. Nothing has been deleted.")
                    
            # create buttons for deleting students
            button_frame = tk.Frame(classlist_frame)
            button_frame.pack(pady=10)

            delete_button = tk.Button(button_frame, text='Delete', command=delete_student)
            delete_button.pack(side=tk.LEFT, padx=5)
                
            # create buttons for adding students
            button_frame = tk.Frame(classlist_frame)
            button_frame.pack(pady=10)
            
            add_button = tk.Button(button_frame, text='Add', command=add_student)
            add_button.pack(side=tk.LEFT, padx=5)
                
                
            
        # create a button to view the classlist
        update_classlist_button = tk.Button(course_window, text="Update Classlist", command=update_classlist)
        update_classlist_button.pack()
            
    else:
        cursor.execute("SELECT * FROM classlist WHERE StudentsName=%s AND RegNo=%s", (username_input.get(), password_input.get()))
        student = cursor.fetchone()
        if student is not None:
            print("Welcome students")
            # create a new window to display students details
            students_window = tk.Toplevel()
            students_window.title("Student Details")
            # create a label to display the students name and RegNo
            students_label = tk.Label(students_window, text="StudentsName: {} \nRegNo: {}".format(student[1], student[0]))
            students_label.pack()
            # Define a function to capture a frame from the webcam and decode any QR codes
            def scan_qr_code():
                # Open the default camera
                cap = cv2.VideoCapture(0)
                # Continuously capture frames from the camera until a QR code is detected
                while True:           
                    # Capture a frame from the camera
                    ret, frame = cap.read()

                    # Decode any QR codes in the frame
                    qr_codes = decode(frame)

                    # If a QR code is detected, return its data
                    if len(qr_codes) > 0:
                        return qr_codes[0].data.decode()
                        # Display the frame in a window
                        cv2.imshow('Scan QR Code', frame)
                # Release the camera and destroy the window
                cap.release()
                cv2.destroyAllWindows()    
            qr_button = tk.Button(students_window, text="Scan QR Code", command=scan_qr_code)
            qr_button.pack()
        else:
            print("Invalid")
            
submit_button = Button(root, text="Submit", command=submit)
submit_button.pack()
     
root.mainloop()
